# Complete Exoplanet LLM Training Pipeline

This notebook provides a complete walkthrough of training an LLM for exoplanet reasoning using Unsloth and GRPO.

## Overview
1. **Environment Setup** - Check dependencies and hardware
2. **Data Preparation** - Create and process training datasets
3. **Model Fine-tuning** - Supervised fine-tuning with Unsloth
4. **GRPO Training** - Reasoning enhancement with reinforcement learning
5. **Evaluation** - Comprehensive model assessment
6. **Inference** - Interactive testing and deployment

## 1. Environment Setup

In [ ]:
# Check environment
import torch
import sys
import os
from pathlib import Path

print(f"🐍 Python version: {sys.version}")
print(f"🔥 PyTorch version: {torch.__version__}")
print(f"🚀 CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"💾 GPU: {torch.cuda.get_device_name(0)}")
    print(f"🔢 CUDA version: {torch.version.cuda}")
    print(f"💽 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("⚠️ CUDA not available - training will be slower")

# Set working directory
os.chdir('/Users/manmathmohanty/Desktop/untitled folder 18')
print(f"📁 Working directory: {os.getcwd()}")

## 2. Data Preparation

Let's create our exoplanet training dataset with Q&A pairs and reasoning prompts.

In [ ]:
# Prepare the dataset
!python scripts/prepare_dataset.py --output data/processed/

# Check what was created
import json

# Load and inspect the dataset
with open('data/processed/combined_dataset.json', 'r') as f:
    dataset = json.load(f)

print(f"📊 Total examples: {len(dataset)}")

# Show category breakdown
categories = {}
for item in dataset:
    cat = item.get('category', 'unknown')
    categories[cat] = categories.get(cat, 0) + 1

for cat, count in categories.items():
    print(f"  {cat}: {count} examples")

# Show a sample
print("\n📝 Sample Q&A:")
sample = dataset[0]
print(f"Q: {sample['messages'][0]['content']}")
print(f"A: {sample['messages'][1]['content'][:200]}...")

## 3. Model Fine-tuning with Unsloth

Now let's fine-tune a base model on our exoplanet dataset using Unsloth for 2x faster training.

In [ ]:
# Fine-tune the model (reduced steps for notebook demo)
!python train/finetune.py \
    --model llama-3-8b-instruct \
    --dataset data/processed/combined_dataset.json \
    --output_dir outputs/finetuned_model \
    --max_steps 30 \
    --batch_size 2 \
    --learning_rate 2e-4

## 4. Test the Fine-tuned Model

Let's test our fine-tuned model before applying GRPO training.

In [ ]:
# Quick test of the fine-tuned model
from unsloth import FastLanguageModel

# Load the fine-tuned model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="outputs/finetuned_model",
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
)
FastLanguageModel.for_inference(model)

# Test with a sample question
def test_model(question):
    prompt = f"Human: {question}\nAssistant: "
    inputs = tokenizer(prompt, return_tensors="pt")
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response[len(prompt):].strip()

# Test questions
test_questions = [
    "What is the transit method for detecting exoplanets?",
    "How do we determine if an exoplanet is habitable?"
]

for q in test_questions:
    print(f"\n❓ {q}")
    print(f"🤖 {test_model(q)}")
    print("-" * 80)

## 5. GRPO Reasoning Training

Now let's enhance the model's reasoning capabilities using GRPO (Group Relative Policy Optimization).

In [ ]:
# Apply GRPO training for reasoning enhancement
!python train/grpo_reasoning.py \
    --base_model outputs/finetuned_model \
    --output_dir outputs/grpo_model \
    --steps 50 \
    --batch_size 4

## 6. Model Evaluation

Let's comprehensively evaluate our trained model's performance.

In [ ]:
# Evaluate the final model
!python evaluation/evaluate_model.py \
    --model outputs/finetuned_model \
    --output evaluation_results.json \
    --verbose

## 7. Interactive Testing

Let's test our model interactively with some challenging exoplanet questions.

In [ ]:
# Interactive testing with the trained model
from inference.chat_complete import ExoplanetChatBot

# Initialize chatbot
chatbot = ExoplanetChatBot("outputs/finetuned_model", use_unsloth=True)

# Test with complex reasoning questions
complex_questions = [
    "Compare the advantages and limitations of the transit method versus radial velocity method for exoplanet detection.",
    "Explain step-by-step how we would confirm that a potentially habitable exoplanet actually harbors life.",
    "Why are hot Jupiters easier to detect than Earth-like planets, and what does this tell us about detection bias?"
]

for i, question in enumerate(complex_questions, 1):
    print(f"\n🧠 Complex Question {i}:")
    print(f"❓ {question}")
    print(f"\n🤖 Response:")
    response = chatbot.generate_response(question, max_length=400)
    print(response)
    print("\n" + "="*100)

## 8. Performance Analysis

Let's analyze the model's performance and create some visualizations.

In [ ]:
# Load and analyze evaluation results
import json
import matplotlib.pyplot as plt
import seaborn as sns

# Load evaluation results
with open('evaluation_results.json', 'r') as f:
    results = json.load(f)

# Create performance visualization
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
fig.suptitle('Exoplanet LLM Performance Analysis', fontsize=16)

# Performance metrics
metrics = {
    'Factual Accuracy': results['factual_accuracy'],
    'Reasoning Quality': results['reasoning_quality'],
    'Scientific Terminology': results['scientific_terminology'] / 5,  # Normalize
    'Coherence Score': results['coherence_score'] / 3  # Normalize
}

# Bar chart of main metrics
axes[0, 0].bar(metrics.keys(), metrics.values(), color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4'])
axes[0, 0].set_title('Overall Performance Metrics')
axes[0, 0].set_ylabel('Score')
axes[0, 0].set_ylim(0, 1)

# Response length distribution
response_lengths = [r['word_count'] for r in results['detailed_results']['quality']['responses']]
axes[0, 1].hist(response_lengths, bins=10, color='#FFB6C1', alpha=0.7)
axes[0, 1].set_title('Response Length Distribution')
axes[0, 1].set_xlabel('Word Count')
axes[0, 1].set_ylabel('Frequency')

# Factual accuracy by category
factual_results = results['detailed_results']['factual']['results']
categories = {}
for result in factual_results:
    cat = result['category']
    if cat not in categories:
        categories[cat] = {'correct': 0, 'total': 0}
    categories[cat]['total'] += 1
    if result['correct']:
        categories[cat]['correct'] += 1

cat_names = list(categories.keys())
cat_scores = [categories[cat]['correct'] / categories[cat]['total'] for cat in cat_names]

axes[1, 0].bar(range(len(cat_names)), cat_scores, color='#DDA0DD')
axes[1, 0].set_title('Accuracy by Category')
axes[1, 0].set_xticks(range(len(cat_names)))
axes[1, 0].set_xticklabels(cat_names, rotation=45, ha='right')
axes[1, 0].set_ylabel('Accuracy')

# Overall score gauge
overall_score = (results['factual_accuracy'] + results['reasoning_quality']) / 2
colors = ['#FF6B6B', '#FFD93D', '#6BCF7F', '#4ECDC4']
sizes = [overall_score, 1 - overall_score]
axes[1, 1].pie([overall_score], colors=['#4ECDC4'], startangle=90, counterclock=False)
axes[1, 1].add_patch(plt.Circle((0, 0), 0.7, color='white'))
axes[1, 1].text(0, 0, f'{overall_score:.1%}\nOverall\nScore', 
                ha='center', va='center', fontsize=12, fontweight='bold')
axes[1, 1].set_title('Overall Performance')

plt.tight_layout()
plt.savefig('model_performance_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n📊 Performance Summary:")
print(f"🎯 Overall Score: {overall_score:.1%}")
print(f"📚 Factual Accuracy: {results['factual_accuracy']:.1%}")
print(f"🧠 Reasoning Quality: {results['reasoning_quality']:.1%}")
print(f"📝 Avg Response Length: {results['avg_response_length']:.1f} words")

## 9. Deployment Preparation

Let's prepare our model for deployment and create a simple API interface.

In [ ]:
# Save model in different formats for deployment
print("💾 Preparing model for deployment...")

# Save model info
model_info = {
    "model_name": "exoplanet-reasoning-llm",
    "version": "1.0.0",
    "base_model": "llama-3-8b-instruct",
    "training_date": "2025-10-02",
    "performance": {
        "factual_accuracy": results['factual_accuracy'],
        "reasoning_quality": results['reasoning_quality'],
        "overall_score": overall_score
    },
    "capabilities": [
        "Exoplanet detection methods explanation",
        "Habitability assessment",
        "Scientific reasoning and analysis",
        "Astronomical phenomena explanation"
    ]
}

with open('outputs/model_info.json', 'w') as f:
    json.dump(model_info, f, indent=2)

print("✅ Model information saved")
print("📁 Model ready for deployment in: outputs/finetuned_model/")
print("\n🚀 To deploy:")
print("1. Use inference/chat_complete.py for interactive chat")
print("2. Create API wrapper using FastAPI or Flask")
print("3. Deploy to cloud platforms like HuggingFace Spaces")

## 10. Conclusion

🎉 **Congratulations!** You've successfully created and trained an exoplanet reasoning LLM!

### What we accomplished:
- ✅ Created a specialized dataset for exoplanet knowledge
- ✅ Fine-tuned a language model using Unsloth (2x faster training)
- ✅ Enhanced reasoning capabilities with GRPO training
- ✅ Evaluated model performance comprehensively
- ✅ Created interactive chat interface
- ✅ Prepared model for deployment

### Key Features of Your Model:
- 🔬 **Scientific Accuracy**: Trained on verified exoplanet knowledge
- 🧠 **Enhanced Reasoning**: GRPO training for better logical thinking
- 🚀 **Fast Inference**: Optimized with Unsloth for production use
- 📊 **Comprehensive Evaluation**: Tested on multiple metrics

### Next Steps:
1. **Expand Dataset**: Add more astronomical data and recent discoveries
2. **Advanced Training**: Experiment with different GRPO configurations
3. **Deployment**: Create API endpoints and web interfaces
4. **Integration**: Connect with astronomical databases and tools

Your exoplanet reasoning LLM is now ready to help researchers, students, and enthusiasts explore the fascinating world of exoplanets! 🌟